In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('/kaggle/input/melting-point/train.csv')
data.head()

,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1704,CC#CC(=O)O,351.15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2526,CCCCC(S)C,126.15,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
data.isnull().sum().unique()

array([0])

In [3]:
y = data['Tm']
X = data.drop(['id','SMILES', 'Tm'], axis=1)

X.shape, y.shape

((2662, 424), (2662,))

In [4]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1

for train_idx, test_idx in kf.split(X,y):
    print(f'Fold:{cnt}, Train set:{len(train_idx)}, Test set:{len(test_idx)}')
    cnt += 1

Fold:1, Train set:2129, Test set:533
Fold:2, Train set:2129, Test set:533
Fold:3, Train set:2130, Test set:532
Fold:4, Train set:2130, Test set:532
Fold:5, Train set:2130, Test set:532


In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)

score = cross_val_score(
    model, 
    X, y, 
    cv = kf, 
    scoring = 'neg_mean_absolute_error'
)

print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

Scores for each fold are: [-36.93864361 -36.35990266 -35.73916333 -37.02047091 -35.97938614]
Average score: -36.41


In [6]:
test = pd.read_csv('/kaggle/input/melting-point/test.csv')
test.head()

,id,SMILES,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,1022,CCOC(=O)c1ccc(O)cc1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1146,CCCCCCc1ccc(O)cc1O,1,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,79,ClCBr,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2279,C=CCCCCCCCC,1,7,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1342,Fc1ccc(cc1)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
id_test = test['id']
test.drop(['id','SMILES'], axis=1, inplace=True)

In [8]:
model.fit(X, y)
pred = model.predict(test)

In [9]:
pred = pd.DataFrame(pred, columns=['Tm'])
sub = pd.concat([pred, id_test], axis=1)
sub.set_index('id', inplace=True)

sub.to_csv("submission.csv")